In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
import pyperclip
from datetime import datetime
from selenium.webdriver.common.alert import Alert
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import json

In [ ]:
#팝업창 제어
#             main = driver.window_handles
#             print("main")
#             print(main)
            
#             for popup in main:
#                 if popup != main[0]:
#                     driver.switch_to.window(popup)
#                     driver.close()
            
             #팝업창 제어끝

In [2]:
cgv_url = 'http://www.cgv.co.kr/user/login/?returnURL=http%3a%2f%2fwww.cgv.co.kr%2fticket%2fdefault.aspx'

In [3]:
def clipboard_input(user_xpath, user_input):
        temp_user_input = pyperclip.paste()  # 사용자 클립보드를 따로 저장

        pyperclip.copy(user_input)
        driver.find_element_by_xpath(user_xpath).click()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

        pyperclip.copy(temp_user_input)  # 사용자 클립보드에 저장 된 내용을 다시 가져 옴
        time.sleep(1)

In [4]:
def current_time():
    now = datetime.now()
    return now.day

#     print(now.date())
#     print("현재 : ", now)
#     now.date().now.strftime('%Y-%m-%d %H:%M:%S'))
#       print("현재 시간 : ", now.time())
#     print("문자열 변환 : ", now.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
# import requests

url = 'https://kauth.kakao.com/oauth/token'
rest_api_key = 'key'
redirect_uri = 'redirect_url'
authorize_code = 'token'


data = {
    'grant_type':'authorization_code',
    'client_id':rest_api_key,
    'redirect_uri':redirect_uri,
    'code': authorize_code,
    }

response = requests.post(url, data=data)
tokens = response.json()

#발행된 토큰 저장
with open("token.json","w") as kakao:
    json.dump(tokens, kakao)

In [64]:
def kakaotalk_messege(messege):
    #발행한 토큰 불러오기
    with open("token.json","r") as kakao:
        tokens = json.load(kakao)

    url="https://kapi.kakao.com/v2/api/talk/memo/default/send"

    headers={
        "Authorization" : "Bearer " + tokens["access_token"]
    }

    data = {
           'object_type': 'text',
           'text': messege,
           'link': {
               'web_url': 'https://developers.kakao.com',
               'mobile_web_url': 'https://developers.kakao.com'
           },
           'button_title': '키워드'
       }

    data = {'template_object': json.dumps(data)}
    response = requests.post(url, headers=headers, data=data)
    response.status_code

In [67]:
def cgvCrolling():

    driver = webdriver.Chrome('D:\chromeDriver\chromedriver_win32\chromedriver.exe')
    driver.get(cgv_url)
    login_click = driver.find_element_by_xpath(f'//*[@id="contents"]/div/div[1]/div/div/a')
    login_click.click()
    time.sleep(1)
    clipboard_input('//*[@id="id_line"]', 'id')
    clipboard_input('//*[@id="pw_line"]', 'password')
    driver.find_element_by_xpath('//*[@id="log.login"]').click()
    driver.find_element_by_xpath('//*[@id="new.save"]').click()
    #로그인 끝 로그인한 cgv홈페이지나옴

    time.sleep(3)
    driver.switch_to.frame('ticket_iframe') ##iframe

    driver.find_element_by_xpath('//*[@id="movie_list"]/ul/li[1]/a').click()#아바타-물의길 클릭
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="sbmt_4dx"]').click()#아바타4DX 클릭

    # ##여기서부터 마우스스크롤
    # scroll = driver.find_element_by_xpath('//*[@id="theater_area_list"]/ul/li[2]/div/div[1]/div')
    # # ActionChains생성
    # action = ActionChains(driver)
    # # 클릭하고 잡기
    # action.click_and_hold(scroll).perform()
    # # 마우스 내리기 
    # action.move_by_offset(0, 300).perform()
    # # 마우스 놓아주기
    # action.release(scroll).perform()
    # ##여기까지 마우스 스크롤 끝
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="theater_area_list"]/ul/li[2]/div/ul/li[10]').click() #용산아이파크몰 클릭

    current_day = current_time() #현재 일

    for i in range(3,22):
        time.sleep(2)
        theater_day_path = driver.find_element_by_xpath(f'//*[@id="date_list"]/ul/div/li[{i}]/a/span[2]')
        select_able_path = driver.find_element_by_xpath(f'//*[@id="date_list"]/ul/div/li[{i}]/a/span[3]')

        theater_day = int(theater_day_path.text)

        if(current_day<theater_day):
            print(theater_day)
            driver.find_element_by_xpath(f'//*[@id="date_list"]/ul/div/li[{i}]/a/span[2]').click() #날짜클릭
            time.sleep(2)
            #경고창처리
            try:
                alert = driver.switch_to.alert
                alert.accept()
                time.sleep(1)
                break
            except:
                pass
            #경고창 처리끝


            for j in range(2,5):

                theater_hour_path = driver.find_element_by_xpath(f'//*[@id="ticket"]/div[2]/div[1]/div[4]/div[2]/div[3]/div[1]/div/ul/li[{j}]/a/span[1]/span') #영화시간
                theater_hour = theater_hour_path.text
    
                driver.find_element_by_xpath(f'//*[@id="ticket"]/div[2]/div[1]/div[4]/div[2]/div[3]/div[1]/div/ul/li[{j}]/a/span[1]').click() #영화시간클릭
                driver.find_element_by_xpath('//*[@id="tnb_step_btn_right"]').click() #예매하기 버튼클릭
                time.sleep(2)

                driver.find_element_by_xpath('/html/body/div[5]/div[3]/a[1]').click()#관람안내 팝업창 확인
                driver.find_element_by_xpath('/html/body/div[4]/div[3]/a').click()#관람안내 팝업창 확인

                driver.find_element_by_xpath('//*[@id="nop_group_adult"]/ul/li[3]/a').click() #일반2명클릭

                for k in range(5,9): ##E F G H I 열
                    for l in range(3,5): # 2분단 3분단
                        input_list = []
                        print(f"{k}열")
                        print(f"{l-1}분단")

                        a = driver.find_element_by_xpath(f'//*[@id="seats_list"]/div[1]/div[{k}]/div[{l}]/div[1]')
                        input_list.append(a.text)
                        input_list = input_list[0].split('\n')
                        if(len(input_list) == 4):
                            print("넘어감")
                            continue
                        else:
                            kakaotalk_messege(f'날짜는 : {theater_day}이고 시간은 : {theater_hour}이고 명당자리찾음!!')



                driver.find_element_by_xpath('//*[@id="ticket_tnb"]/div/a[1]').click() ##뒤로가기
                time.sleep(1)


        










In [1]:
cgvCrolling()